In [ ]:
# Copyright 2019 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

<img src="http://developer.download.nvidia.com/compute/machine-learning/frameworks/nvidia_logo.png" style="width: 90px; float: right;">

# End-to-End BERT (Inference)

이 문서는 TensorFlow BERT pretrained weight를 TensorRT engine으로 변환한 이후, TRTIS에 연동해서 Inference Serving하는 방법을 안내하기 위해 작성이 되었습니다. 이 문서에서 처리하는 절차는 크게 다음 두 가지 입니다.

**1. BERT TensorRT inference engine build**
<img src="https://developer.nvidia.com/sites/default/files/akamai/deeplearning/tensorrt/trt-info.png" width="600" />
TensorFlow이용하여 학습된 BERT pretrained weight를 TensorRT engine 파일로 변환합니다. 이 예제에서는 이 과정에서 필요한 plugin 들을 build하는 과정도 포함합니다. TensorRT를 이용한 BERT Sample에 대한 자세한 설명을 보시려면, [Real-Time Natural Language Understanding with BERT Using TensorRT](https://devblogs.nvidia.com/nlu-with-tensorrt-bert/)를 참고하세요.

**2. TRTIS model repository 구성 및 TRTIS 서버 실행**
<img src="https://developer.nvidia.com/sites/default/files/pictures/2018/trt-inference-server-diagram-1200px.png" width="600" />
이 예제에서는 완성된 engine 파일을 이용하여, TRTIS용 Model repository를 구성하고, TensorRT Inference Server를 실제로 구동하여 동작하는 것을 살펴볼 것입니다. TensorRT Inference Server에 대한 자세한 설명을 보시려면, [NVIDIA TensorRT Inference Server Boosts Deep Learning Inference](https://devblogs.nvidia.com/nvidia-serves-deep-learning-inference/) 문서를 참고하세요.

이 문서는 DGX-1 V100 16GB 장비를 이용하여 테스트 되었으며, 구동 환경에 따라 성능은 다를 수 있습니다. 또한 사용된 SW는 다음과 같습니다.
* CUDA 10.1 / CUDNN 7 / TensorRT 6.0
* NGC Containers

| 용도 | NGC container |
|:---:|:---:|
| TensorRT engine build | cuda:10.1-cudnn7-devel-ubuntu18.04 |
| TensorRT Inference Server | tensorrtserver:12.10-py3 |
| BERT inference client | tensorflow:12.08-py3 |
    
* docker / nvidia-docker2

## I. Building BERT TensorRT Inference Engine

### 1. Build TensorRT Docker Container

TensorRT engine을 build하기 위해 우선 build 환경을 구축하기 위한 docker image를 생성합니다. 여기서는 TensorRT 6.0 BERT inference 예제를 이용할 것이며, 기준 환경인 CUDA 10.1 / TensorRT 6.0을 이용할 것입니다.

In [1]:
%%bash
cd ../trt
docker build . -f Dockerfile -t bert_trt --rm \
    --build-arg FROM_IMAGE_NAME=nvcr.io/nvidia/cuda:10.1-cudnn7-devel-ubuntu18.04 \
    --build-arg TRT_PKG_VERSION=6.0.1-1+cuda10.1 \
    --build-arg myuid=$(id -u) --build-arg mygid=$(id -g) > /dev/null 2>&1

이 문서에서는 편의상 docker image를 build 하면서 생기는 log를 숨겼습니다. 자세한 로그를 확인하고 싶으시다면 위 코드에서 ```> /dev/null 2>&1```을 제거하고 코드를 실행하시면 됩니다.

한편 이 이미지를 빌드하는 과정에서 TensorRT 6.0의 repository를 clone하여, BERT TensorRT plugin layer들을 공유 라이브러리 파일 형태로 구성합니다. 이 과정을 위해 Dockerfile에 아래의 코드가 삽입되어 있습니다. 

```bash
RUN git clone -b release/6.0 https://github.com/nvidia/TensorRT TensorRT && \
   cd TensorRT && \
   git submodule update --init --recursive && \
   mkdir -p demo/BERT/build && cd demo/BERT/build && \
   cmake .. && \
   make -j$(nproc)
```

그리고 이렇게 build한 라이브러리를 TensorRT Inference Server에서 가져다 사용하기 위해 아래의 명령어로 압축을 합니다.

```bash
tar -czf bert_plugin.tar.gz libbert_plugins.so libcommon.so
```

자세한 내용은 [```trt/Dockerfile```](../trt/Dockerfile)을 참고해주시기 바랍니다.

### 2. Container 실행

다음 docker 실행 명령을 이용하여 BERT TensorRT engine을 build하기 위한 container를 실행합니다.

In [2]:
%%bash
cd ..

GPU_ID=${1:-"all"}

ENGINE_OUTPUT_DIR="outputs"

if [[ ! -e ${ENGINE_OUTPUT_DIR} ]]; then
    mkdir -p ${ENGINE_OUTPUT_DIR}
fi

docker rm -f bert_trt
docker run -d -ti \
    --name bert_trt${VERSION} \
    --gpus ${GPU_ID} \
    --shm-size=1g --ulimit memlock=1 --ulimit stack=67108864 \
    -u $(id -u):$(id -g) \
    -v $(pwd)/outputs:/workspace/outputs \
    -v $(pwd)/results/models:/workspace/models \
    bert_trt bash

c14963031837e80d0b4c557b8aa4c70a1148551dac9341dd58fc12c07219ff9b


Error: No such container: bert_trt


In [3]:
!docker ps -a

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS                  PORTS               NAMES
c14963031837        bert_trt            "bash"              1 second ago        Up Less than a second                       bert_trt


앞에서 docker image build 과정에서 설명한 것과 같이, ```/workspace/TensorRT/demo/BERT/build```경로에는 BERT TensorRT plugin이 빌드되어 있습니다.

In [4]:
!docker exec -t bert_trt ls /workspace/TensorRT/demo/BERT/build

### 3. Build TensorRT Plugin Layer library and download pretrained weights

BERT TensorRT engine을 build하기 위해 필요한 plugin의 라이브러리와 예제를 위해 pretrained-weight를 ngc로부터 다운로드 받습니다.

#### 2. Pre-trained weight download
NGC로부터 pre-trained weight를 다운로드 받습니다. NGC에서는 다음의 조건에 대한 pretrained weight를 제공하므로 이 중에 선택하여 사용할 수 있습니다.

| | options |
|:---:|:---:|
| model | large, base |
| precision | fp32, fp16 |
| seq. length | 128, 384 |

물론 독자적으로 학습하신 weight (ckpt)를 사용하실 수도 있습니다.

이 예제에서는 bert-large, fp16, seq-len 128 을 사용하도록 하겠습니다.

In [5]:
%%bash -s 'base' 'fp16' '128'

MODEL=${1:-'base'}
FT_PRECISION=${2:-'fp16'}
SEQ_LEN=${3:-'128'}

if [[ ! -d ../results/models/fine-tuned/bert_tf_v2_${MODEL}_${FT_PRECISION}_${SEQ_LEN}_v2/ ]]; then
    docker exec -t bert_trt bash /workspace/TensorRT/demo/BERT/python/download_fine-tuned_model.sh ${MODEL} ${FT_PRECISION} ${SEQ_LEN}
fi

NGC에서 다운로드 받은 pretrained weight는 ```results/models/fine-tuned/```에 저장이 됩니다.

In [6]:
%%bash -s 'base' 'fp16' '128'

MODEL=${1:-'base'}
FT_PRECISION=${2:-'fp16'}
SEQ_LEN=${3:-'128'}

ls ../results/models/fine-tuned/bert_tf_v2_${MODEL}_${FT_PRECISION}_${SEQ_LEN}_v2/

bert_config.json
model.ckpt-8144.data-00000-of-00001
model.ckpt-8144.index
model.ckpt-8144.meta
tf_bert_squad_1n_fp16_gbs32.190523114449.log
vocab.txt


### 4. Build TensorRT Engine

이제 BERT TensorRT precision을 build할 것입니다. 이전에 NGC model repository에서 다운로드 받은 pretrained weight와 동일한 조건으로 engine을 build하되 batch size를 지정해 줘야 합니다. 현재 TensorRT Inference Server와 호환성 문제로 batch size 1까지만을 지원하므로, 여기서는 batch size를 1로 설정해서 engine을 빌드하도록 하겠습니다.

참고로 별도의 터미널 창을 열어서 ```watch -n1 nvidia-smi```를 이용하면, TensorRT가 engine을 Build하면서 GPU를 점유하는 것을 보실 수 있습니다. 이 과정에서 GPU를 사용하는 이유는 engine을 build하는 과정에서 target GPU의 성능을 측정하여 적절한 GPU Kernel의 구성을 TensorRT가 찾기 때문입니다.

In [7]:
%%time
%%bash -s 'base' 'fp16' '128' '1' 

MODEL=${1:-'base'}
FT_PRECISION=${2:-'fp16'}
SEQ_LEN=${3:-'128'}
BATCH_SIZE=${4:-'1'}

docker exec -t \
    bert_trt \
        python3 -W ignore /workspace/TensorRT/demo/BERT/python/bert_builder.py \
            -m /workspace/models/fine-tuned/bert_tf_v2_${MODEL}_${FT_PRECISION}_${SEQ_LEN}_v2/model.ckpt-8144 \
            -c /workspace/models/fine-tuned/bert_tf_v2_${MODEL}_${FT_PRECISION}_${SEQ_LEN}_v2 \
            -o /workspace/outputs/bert_${MODEL}_${SEQ_LEN}.engine \
            -s ${SEQ_LEN} -b ${BATCH_SIZE}

[TensorRT] INFO: Using configuration file: /workspace/models/fine-tuned/bert_tf_v2_base_fp16_128_v2/bert_config.json
[TensorRT] INFO: Found 202 entries in weight map
[TensorRT] INFO: Detected 3 inputs and 1 output network tensors.
[TensorRT] INFO: Detected 3 inputs and 1 output network tensors.
[TensorRT] INFO: Detected 3 inputs and 1 output network tensors.
[TensorRT] INFO: Saving Engine to /workspace/outputs/bert_base_128.engine
[TensorRT] INFO: Done.
CPU times: user 20 ms, sys: 8.5 ms, total: 28.5 ms
Wall time: 7min 58s


위 Script의 실행결과 아래 경로에 engine 파일이 생성된 것을 볼 수 있습니다.

In [8]:
!docker exec -ti bert_trt ls /workspace/outputs

bert_base_128.engine


### 5. Inference Test

이제 build 한 TensorRT engine을 이용해서 inference를 테스트해보겠습니다.

질문의 내용을 바꿔가면서 답변이 바뀌는 것들을 보실 수 있습니다.

#### Example 1. Introductin TensorRT

In [9]:
%%bash -s 'base' 'fp16' '128' '1'

MODEL=${1:-'base'}
FT_PRECISION=${2:-'fp16'}
SEQ_LEN=${3:-'128'}
BATCH_SIZE=${4:-'1'}

paragraph_text="TensorRT is a high performance deep learning inference platform that delivers low latency and high throughput for apps such as recommenders, speech and image/video on NVIDIA GPUs. It includes parsers to import models, and plugins to support novel ops and layers before applying optimizations for inference. Today NVIDIA is open sourcing parsers and plugins in TensorRT so that the deep learning community can customize and extend these components to take advantage of powerful TensorRT optimizations for your apps."
question_text="What is TensorRT?"

docker exec -t \
    bert_trt \
        python3 -W ignore /workspace/TensorRT/demo/BERT/python/bert_inference.py \
            -e /workspace/outputs/bert_${MODEL}_${SEQ_LEN}.engine -s ${SEQ_LEN} \
            -p ${paragraph_text} \
            -q ${question_text} \
            -v /workspace/models/fine-tuned/bert_tf_v2_${MODEL}_${FT_PRECISION}_${SEQ_LEN}_v2/vocab.txt


Passage: TensorRT is a high performance deep learning inference platform that delivers low latency and high throughput for apps such as recommenders, speech and image/video on NVIDIA GPUs. It includes parsers to import models, and plugins to support novel ops and layers before applying optimizations for inference. Today NVIDIA is open sourcing parsers and plugins in TensorRT so that the deep learning community can customize and extend these components to take advantage of powerful TensorRT optimizations for your apps.


Question: What is TensorRT?

Running Inference...
------------------------
Running inference in 214.017 Sentences/Sec
------------------------
Processing output 0 in batch
Answer: 'high performance deep learning inference platform'
With probability: 29.250


#### Example 2. Apollo program

In [10]:
%%bash -s 'base' 'fp16' '128' '1'

MODEL=${1:-'base'}
FT_PRECISION=${2:-'fp16'}
SEQ_LEN=${3:-'128'}
BATCH_SIZE=${4:-'1'}

paragraph_text="The Apollo program was the third United States human spaceflight program. First conceived as a three-man spacecraft to follow the one-man Project Mercury which put the first Americans in space, Apollo was dedicated to President John F. Kennedy's national goal of landing a man on the Moon. The first manned flight of Apollo was in 1968. Apollo ran from 1961 to 1972 followed by the Apollo-Soyuz Test Project a joint Earth orbit mission with the Soviet Union in 1975."
question_text="What project put the first Americans into space?"
#question_text="What year did the first manned Apollo flight occur?"
#question_text="What President is credited with the original notion of putting Americans in space?"
#question_text="Who did the U.S. collaborate with on an Earth orbit mission in 1975?"

docker exec -t \
    bert_trt \
        python3 -W ignore /workspace/TensorRT/demo/BERT/python/bert_inference.py \
            -e /workspace/outputs/bert_${MODEL}_${SEQ_LEN}.engine -s ${SEQ_LEN} \
            -p ${paragraph_text} \
            -q ${question_text} \
            -v /workspace/models/fine-tuned/bert_tf_v2_${MODEL}_${FT_PRECISION}_${SEQ_LEN}_v2/vocab.txt


Passage: The Apollo program was the third United States human spaceflight program. First conceived as a three-man spacecraft to follow the one-man Project Mercury which put the first Americans in space, Apollo was dedicated to President John F. Kennedy's national goal of landing a man on the Moon. The first manned flight of Apollo was in 1968. Apollo ran from 1961 to 1972 followed by the Apollo-Soyuz Test Project a joint Earth orbit mission with the Soviet Union in 1975.


Question: What project put the first Americans into space?

Running Inference...
------------------------
Running inference in 213.147 Sentences/Sec
------------------------
Processing output 0 in batch
Answer: 'Project Mercury'
With probability: 55.754


예제의 체감 실행 시간과 ```bert_inference.py```에서 측정한 inference 시간에서 차이가 발생하는 이유는, 이 예제는 매번 실행할 때마다 TensorRT 엔진 파일을 Deserialize하기 때문입니다. 실제로 inference를 할 때에는, BERT Inference Engine을 GPU Instance로 띄워둔 상태로 서비스를 하게 되므로 서비스때의 속도는 측정된 시간의 속도대로 동작하게 됩니다.

### 6. Closing the container

이제 TensorRT 엔진을 빌드하는 과정을 마쳤습니다. 그럼 이제 지금까지 사용한 Container를 닫도록 하겠습니다.

In [11]:
!docker rm -f bert_trt

bert_trt


## Building BERT inferencing platform with TRTIS

### 1. Pulling TRTIS docker image

TensorRT Inference Server는 새로운 Build 없이 Serving을 할 수 있는 장점이 있습니다. 우선 원활한 예제의 실행을 위해 사용할 이미지를 다음 명령을 이용하여 pull 합니다.

In [12]:
%%bash
docker pull nvcr.io/nvidia/tensorrtserver:19.10-py3 > /dev/null 2>&1

In [13]:
%%file ../trtis/Dockerfile

ARG FROM_IMAGE_NAME
FROM ${FROM_IMAGE_NAME}

ENV LIB_WORKING_DIR="/opt/tensorrtserver/lib"
COPY --from=bert_trt /workspace/TensorRT/demo/BERT/build/bert_plugin.tar.gz ${LIB_WORKING_DIR}
RUN tar -xzvf ${LIB_WORKING_DIR}/bert_plugin.tar.gz -C ${LIB_WORKING_DIR} && \
    chown -R tensorrt-server:tensorrt-server ${LIB_WORKING_DIR}
    
    

Overwriting ../trtis/Dockerfile


In [14]:
%%bash
docker build ../trtis -t tensorrtserver:bert --build-arg FROM_IMAGE_NAME=nvcr.io/nvidia/tensorrtserver:19.10-py3 > /dev/null 2>&1

### 2. Setting TRTIS model repository

다음의 명령들을 이용하여 TRTIS model repository를 구성합니다. 여기서 숫자 1은 model version으로 원하는 버전을 설정하실 수 있으며, 향후에 inference client 단에서 원하는 버전을 지정하여 inference가 되도록 지정하실 수 있습니다.

In [15]:
%%bash
mkdir -p ../results/trtis_models/bert_base_128_fp16
mkdir -p ../results/trtis_models/bert_base_128_fp16/1

Model repository에는 model를 명시하는 ```config.pbtxt``` 파일과 TensorRT engine 파일을 ```model.plan```으로 이름을 변경하여 버전에 따라 저장을 합니다.

In [31]:
%%file ../results/trtis_models/bert_base_128_fp16/config.pbtxt

name: "bert_base_128_fp16"
platform: "tensorrt_plan"
max_batch_size: 1

instance_group [
    {
        count: 1
        kind: KIND_GPU
        gpus: [0]
    }
]


Overwriting ../results/trtis_models/bert_base_128_fp16/config.pbtxt


In [17]:
%%bash
cp ../outputs/bert_base_128.engine ../results/trtis_models/bert_base_128_fp16/1/model.plan

### 3. Launch Server

이제 TensorRT inference server를 구동시킬 차례입니다. TensorFlow Model이 FP16으로 Inference 되도록 하게 하는 한편, TensorRT engine의 dependency를 위한 경로를 설정해 줍니다.

In [39]:
%%bash -s "fp16"

cd ..

precision=${1:-"fp16"}
NV_VISIBLE_DEVICES=${NVIDIA_VISIBLE_DEVICES:-"all"}

if [ "$precision" = "fp16" ] ; then
   echo "fp16 activated!"
   export TF_ENABLE_AUTO_MIXED_PRECISION_GRAPH_REWRITE=1
else
   echo "fp32 activated!"
   export TF_ENABLE_AUTO_MIXED_PRECISION_GRAPH_REWRITE=0
fi

# Start TRTIS server in detached state
docker run -d --rm \
   --gpus ${NV_VISIBLE_DEVICES} \
   --shm-size=1g \
   --ulimit memlock=-1 \
   --ulimit stack=67108864 \
   -p8000:8000 \
   -p8001:8001 \
   -p8002:8002 \
   --name trt_server_cont \
   -e TF_ENABLE_AUTO_MIXED_PRECISION_GRAPH_REWRITE \
   -v $(pwd)/results/trtis_models:/models \
   -e LD_PRELOAD="libcommon.so:libbert_plugins.so" \
   tensorrtserver:bert \
        trtserver --model-store=/models --strict-model-config=false

fp16 activated!
f0e40d345bea8340b516bb1138d6e225af53ef53ac02c3b9b996fa696bcd7356


### 4. Performance Test

이제 구성한 TensorRT Inference Server의 성능을 확인하기 위해서 TensorRT Inference Server에서 제공하는 [perf_client](https://docs.nvidia.com/deeplearning/sdk/tensorrt-inference-server-guide/docs/perf_client.html)을 이용해서 성능을 측정해보도록 하겠습니다. 이 툴은 

#### 1. BERT Client build
만약 BERT Training을 위해 사용한 BERT docker image가 위치한 노드와 동일한 노드라면 다음 명령은 생략하셔도 됩니다.

In [33]:
%%bash 
cd ..
bash ./scripts/docker/build.sh > /dev/null 2>&1

#### 2. BERT TRTIS performance (1 GPU)

In [34]:
%%bash
SERVER_URI=${1:-"localhost"}

echo "Waiting for TRTIS Server to be ready at http://$SERVER_URI:8000..."

live_command="curl -m 1 -L -s -o /dev/null -w %{http_code} http://$SERVER_URI:8000/api/health/live"
ready_command="curl -m 1 -L -s -o /dev/null -w %{http_code} http://$SERVER_URI:8000/api/health/ready"

current_status=$($live_command)

# First check the current status. If that passes, check the json. If either fail, loop
while [[ ${current_status} != "200" ]] || [[ $($ready_command) != "200" ]]; do

   printf "."
   sleep 1
   current_status=$($live_command)
done

echo "TRTIS Server is ready!"

Waiting for TRTIS Server to be ready at http://localhost:8000...
TRTIS Server is ready!


In [35]:
%%bash -s "base" "128" "fp16" "1" "1" "500" "10" "10" "localhost"

MODEL=${1:-"base"}
SEQ_LEN=${2:-"128"}
FT_PRECISION=${3:-"fp16"}
BATCH_SIZE=${4:-1}
MODEL_VERSION=${5:-1}
MAX_LATENCY=${6:-500}
MAX_CLIENT_THREADS=${7:-10}
MAX_CONCURRENCY=${8:-50}
SERVER_HOSTNAME=${9:-"localhost"}

MODEL_NAME="bert_${MODEL}_${SEQ_LEN}_${FT_PRECISION}"
NV_VISIBLE_DEVICES=${NVIDIA_VISIBLE_DEVICES:-"all"}

if [[ $SERVER_HOSTNAME == *":"* ]]; then
  echo "ERROR! Do not include the port when passing the Server Hostname. These scripts require that the TRTIS HTTP endpoint is on Port 8000 and the gRPC endpoint is on Port 8001. Exiting..."
  exit 1
fi

if [[ ! -e ../results/perf_client/${MODLE_NAME} ]]; then
    mkdir ../results/perf_client/${MODEL_NAME}
fi

TIMESTAMP=$(date "+%y%m%d_%H%M")
OUTPUT_FILE_CSV="/results/perf_client/${MODEL_NAME}/results_${TIMESTAMP}.csv"

cd ..
docker run --rm -t \
    --net=host \
    --shm-size=1g \
    --ulimit memlock=-1 \
    --ulimit stack=67108864 \
    -e NVIDIA_VISIBLE_DEVICES=$NV_VISIBLE_DEVICES \
    -u $(id -u):$(id -g) \
    -v $(pwd):/workspace/bert \
    -v $(pwd)/results:/results \
    bert \
        /workspace/install/bin/perf_client \
            --max-threads ${MAX_CLIENT_THREADS} \
            -m ${MODEL_NAME} \
            -x ${MODEL_VERSION} \
            -p 3000 \
            -d \
            -v \
            -i gRPC \
            -u ${SERVER_HOSTNAME}:8001 \
            -b ${BATCH_SIZE} \
            -l ${MAX_LATENCY} \
            -c ${MAX_CONCURRENCY} \
            -f ${OUTPUT_FILE_CSV} \
            -z

                                                                                                                                                
== TensorFlow ==

NVIDIA Release 19.08 (build 7791926)
TensorFlow Version 1.14.0

Container image Copyright (c) 2019, NVIDIA CORPORATION.  All rights reserved.
Copyright 2017-2019 The TensorFlow Authors.  All rights reserved.

Various files include modifications (c) NVIDIA CORPORATION.  All rights reserved.
NVIDIA modifications are covered by the license terms that apply to the underlying project or file.

   Use 'nvidia-docker run' to start this container; see
   https://github.com/NVIDIA/nvidia-docker/wiki/nvidia-docker .

ERROR: Detected MOFED driver 4.7-1.0.0, but this container has version 4.4-1.0.0.
       Unable to automatically upgrade this container.
       Use of RDMA for multi-node communication will be unreliable.

NOTE: MOFED driver was detected, but nv_peer_mem driver was not detected.
      Multi-node communication performance m

#### 3. Model reconfiguration & updated inference performance - 2 GPU

GPU의 구성을 바꾸기 위해 model repository에 있는 GPU 정보를 새롭게 업데이트 해줍니다.

In [36]:
%%file ../results/trtis_models/bert_base_128_fp16/config.pbtxt

name: "bert_base_128_fp16"
platform: "tensorrt_plan"
max_batch_size: 1

instance_group [
    {
        count: 0
        kind: KIND_GPU
        gpus: [0, 1]
    }
]


Overwriting ../results/trtis_models/bert_base_128_fp16/config.pbtxt


그러면 TensorRT Inference Server에서는 이를 감지하여 새로 GPU instance를 구성합니다.

In [40]:
%%bash
SERVER_URI=${1:-"localhost"}

echo "Waiting for TRTIS Server to be ready at http://$SERVER_URI:8000..."

live_command="curl -m 1 -L -s -o /dev/null -w %{http_code} http://$SERVER_URI:8000/api/health/live"
ready_command="curl -m 1 -L -s -o /dev/null -w %{http_code} http://$SERVER_URI:8000/api/health/ready"

current_status=$($live_command)

# First check the current status. If that passes, check the json. If either fail, loop
while [[ ${current_status} != "200" ]] || [[ $($ready_command) != "200" ]]; do

   printf "."
   sleep 1
   current_status=$($live_command)
done

echo "TRTIS Server is ready!"

Waiting for TRTIS Server to be ready at http://localhost:8000...
.........TRTIS Server is ready!


이 시점에서 nvidia-smi 등을 통해서 GPU 메모리 사용량을 보시면, 2개의 GPU에 메모리 사용량이 늘어난 것을 보실 수 있습니다.

이제 테스트를 해볼 차례입니다. 다만 여기서는 Client 단의 max latency를 기존의 500에서 1000으로 조정하도록 하겠습니다.

In [41]:
%%bash -s "base" "128" "fp16" "1" "1" "1000" "10" "10" "localhost"

MODEL=${1:-"base"}
SEQ_LEN=${2:-"128"}
FT_PRECISION=${3:-"fp16"}
BATCH_SIZE=${4:-1}
MODEL_VERSION=${5:-1}
MAX_LATENCY=${6:-500}
MAX_CLIENT_THREADS=${7:-10}
MAX_CONCURRENCY=${8:-50}
SERVER_HOSTNAME=${9:-"localhost"}

MODEL_NAME="bert_${MODEL}_${SEQ_LEN}_${FT_PRECISION}"
NV_VISIBLE_DEVICES=${NVIDIA_VISIBLE_DEVICES:-"all"}

if [[ $SERVER_HOSTNAME == *":"* ]]; then
  echo "ERROR! Do not include the port when passing the Server Hostname. These scripts require that the TRTIS HTTP endpoint is on Port 8000 and the gRPC endpoint is on Port 8001. Exiting..."
  exit 1
fi

if [[ ! -e ../results/perf_client/${MODLE_NAME} ]]; then
    mkdir ../results/perf_client/${MODEL_NAME}
fi

TIMESTAMP=$(date "+%y%m%d_%H%M")
OUTPUT_FILE_CSV="/results/perf_client/${MODEL_NAME}/results_${TIMESTAMP}.csv"

cd ..
docker run --rm -t \
    --net=host \
    --shm-size=1g \
    --ulimit memlock=-1 \
    --ulimit stack=67108864 \
    -e NVIDIA_VISIBLE_DEVICES=$NV_VISIBLE_DEVICES \
    -u $(id -u):$(id -g) \
    -v $(pwd):/workspace/bert \
    -v $(pwd)/results:/results \
    bert \
        /workspace/install/bin/perf_client \
            --max-threads ${MAX_CLIENT_THREADS} \
            -m ${MODEL_NAME} \
            -x ${MODEL_VERSION} \
            -p 3000 \
            -d \
            -v \
            -i gRPC \
            -u ${SERVER_HOSTNAME}:8001 \
            -b ${BATCH_SIZE} \
            -l ${MAX_LATENCY} \
            -c ${MAX_CONCURRENCY} \
            -f ${OUTPUT_FILE_CSV} \
            -z

                                                                                                                                                
== TensorFlow ==

NVIDIA Release 19.08 (build 7791926)
TensorFlow Version 1.14.0

Container image Copyright (c) 2019, NVIDIA CORPORATION.  All rights reserved.
Copyright 2017-2019 The TensorFlow Authors.  All rights reserved.

Various files include modifications (c) NVIDIA CORPORATION.  All rights reserved.
NVIDIA modifications are covered by the license terms that apply to the underlying project or file.

   Use 'nvidia-docker run' to start this container; see
   https://github.com/NVIDIA/nvidia-docker/wiki/nvidia-docker .

ERROR: Detected MOFED driver 4.7-1.0.0, but this container has version 4.4-1.0.0.
       Unable to automatically upgrade this container.
       Use of RDMA for multi-node communication will be unreliable.

NOTE: MOFED driver was detected, but nv_peer_mem driver was not detected.
      Multi-node communication performance m

축하합니다. 이제 BERT 모델을 GPU를 이용하여 최적의 성능으로 Inference를 하실 수 있게 되셨습니다.

### 5. Closing TRTIS container

이제 아래 명령을 통해 실행한 container를 종료합니다.

In [30]:
!docker rm -f trt_server_cont

trt_server_cont


<img src="http://developer.download.nvidia.com/compute/machine-learning/frameworks/nvidia_logo.png" style="width: 150px; float: center;">